
# Paso a paso: EDA → Preparación → Modelo (HISTORICO_SUERTES)

Este cuaderno te guía con **código paso a paso** desde la exploración hasta el modelo final.  
Dataset base: `/mnt/data/HISTORICO_SUERTES.xlsx`  
(Integración opcional: `/mnt/data/BD_IPSA_1940.xlsx` si existe)


## 0) Configuración y librerías

In [ ]:

from __future__ import annotations

import os, warnings, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, learning_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

warnings.filterwarnings("ignore")

DATA_PATH = "/mnt/data/HISTORICO_SUERTES.xlsx"
DATA_PATH_IPSA = "/mnt/data/BD_IPSA_1940.xlsx"  # opcional
RANDOM_STATE = 42
TEST_SIZE = 0.2
N_FOLDS = 5
SCALER = "standard"  # "standard" | "minmax" | None
OUTPUT_DIR = "outputs_step"
Path(OUTPUT_DIR).mkdir(exist_ok=True, parents=True)
print("OK configuración")

## 1) Carga de datos

In [ ]:

assert os.path.exists(DATA_PATH), f"No se encontró el archivo: {DATA_PATH}"
df = pd.read_excel(DATA_PATH)
print("Shape:", df.shape)
df.head()

## 2) Panorama general y diccionario inferido

In [ ]:

display(df.sample(min(5, len(df))))
df.info()
display(df.describe(include='all').T)

dtypes = df.dtypes.astype(str).rename("dtype")
nunique = df.nunique().rename("nunique")
n_missing = df.isna().sum().rename("n_missing")
pct_missing = (df.isna().mean()*100).round(2).rename("%missing")
example = df.head(2).T.iloc[:, :1].rename(columns={df.head(2).columns[0]: "example"})
data_dict = pd.concat([dtypes, nunique, n_missing, pct_missing, example], axis=1).sort_index()
data_dict.to_csv(os.path.join(OUTPUT_DIR, "data_dictionary.csv"))
display(data_dict.head(30))

## 3) Faltantes y duplicados

In [ ]:

missing_tbl = pd.DataFrame({"n_missing": df.isna().sum(), "%missing": (df.isna().mean()*100).round(2)}).sort_values("%missing", ascending=False)
missing_tbl.to_csv(os.path.join(OUTPUT_DIR, "missing_summary.csv"))
display(missing_tbl.head(30))

dups = df.duplicated().sum()
print("Duplicados:", dups)
if dups>0:
    df = df.drop_duplicates().reset_index(drop=True)
    print("Eliminados duplicados. Nuevo shape:", df.shape)

## 4) Casting y tratamiento de fechas (heurístico)

In [ ]:

date_like_cols = [c for c in df.columns if any(x in str(c).lower() for x in ["f.siembra","siembra","fec.madur","madur","ult.riego","f.ult.corte","fecha"])]
for c in date_like_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
print("Columnas tratadas como fecha:", date_like_cols)

## 5) Outliers (IQR y Z-score)

In [ ]:

def iqr_bounds(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
out_rows = []
for c in num_cols_all:
    s = df[c].dropna()
    if len(s) < 3: 
        continue
    low, up = iqr_bounds(s)
    iqr_frac = ((s<low)|(s>up)).mean()
    mu, sd = s.mean(), s.std(ddof=0)
    z_frac = (sd>0) and ((np.abs((s-mu)/sd)>3).mean()) or 0.0
    out_rows.append((c, round(iqr_frac*100,2), round(z_frac*100,2)))
out_df = pd.DataFrame(out_rows, columns=["col","%outliers_IQR","%outliers_Z>3"]).sort_values("%outliers_IQR", ascending=False)
display(out_df.head(30))

## 6) Univariado (numérico y categórico)

In [ ]:

# Histogramas numéricos (sin estilos ni colores específicos)
for c in num_cols_all[:10]:  # muestra hasta 10 para agilidad
    plt.figure()
    df[c].dropna().hist(bins=30)
    plt.title(f"Histograma: {c}")
    plt.xlabel(c); plt.ylabel("Frecuencia")
    plt.show()

# Barras categóricas top 20 (si existen)
cat_cols_all = df.select_dtypes(include=["object","category","bool"]).columns.tolist()
for c in cat_cols_all[:5]:  # muestra hasta 5 para agilidad
    plt.figure()
    df[c].value_counts(dropna=False).head(20).plot(kind="bar")
    plt.title(f"Frecuencias: {c}")
    plt.xlabel(c); plt.ylabel("Conteo")
    plt.show()

## 7) Correlaciones con objetivos (TCH y sacarosa/%Sac.Caña si existen)

In [ ]:

TARGET_CANDIDATES = ["TCH", "%Sac.Caña", "%Sac.Cana", "% Sac.Caña", "sacarosa"]
present_targets = [t for t in TARGET_CANDIDATES if t in df.columns]
print("Objetivos detectados:", present_targets)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for tgt in present_targets:
    cols = [c for c in num_cols if c != tgt]
    if cols:
        corr = df[[tgt] + cols].corr(numeric_only=True)[tgt].sort_values(ascending=False)
        display(pd.DataFrame(corr).rename(columns={tgt:"corr"}).head(30))

## 8) Ingeniería de variables derivadas (duraciones y densidades)

In [ ]:

df_feat = df.copy()

# Duraciones desde fechas (en días)
col_siembra = next((c for c in df_feat.columns if "siembra" in str(c).lower()), None)
col_ult_corte = next((c for c in df_feat.columns if "ult.corte" in str(c).lower()), None)
col_ult_riego = next((c for c in df_feat.columns if "ult.riego" in str(c).lower()), None)
col_madur = next((c for c in df_feat.columns if "madur" in str(c).lower()), None)

if col_siembra and col_ult_corte:
    df_feat["feat_dias_siembra_a_ultcorte"] = (df_feat[col_ult_corte] - df_feat[col_siembra]).dt.days
if col_siembra and col_madur:
    df_feat["feat_dias_siembra_a_madur"] = (df_feat[col_madur] - df_feat[col_siembra]).dt.days
if col_ult_riego and col_ult_corte:
    df_feat["feat_dias_ult_riego_a_ultcorte"] = (df_feat[col_ult_corte] - df_feat[col_ult_riego]).dt.days

# TCH/Área si ambas existen
col_area = next((c for c in df_feat.columns if "area neta" in str(c).lower() or "área neta" in str(c).lower()), None)
if col_area and "TCH" in df_feat.columns:
    safe_area = df_feat[col_area].replace({0: np.nan})
    df_feat["feat_tch_por_area"] = df_feat["TCH"] / safe_area

# Semanas maduración (si existe)
col_sem_mad = next((c for c in df_feat.columns if "semanas" in str(c).lower() and "mad" in str(c).lower()), None)
if col_sem_mad:
    df_feat["feat_sem_mad"] = pd.to_numeric(df_feat[col_sem_mad], errors="coerce")

[c for c in df_feat.columns if c.startswith("feat_")]

## 9) Preparación: train/test y ColumnTransformer

In [ ]:

def build_preprocess(X, scaler_kind="standard"):
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X.select_dtypes(include=["object","category","bool"]).columns.tolist()

    if scaler_kind == "standard":
        scaler = StandardScaler()
    elif scaler_kind == "minmax":
        scaler = MinMaxScaler()
    else:
        scaler = None

    num_steps = [("imputer", SimpleImputer(strategy="median"))]
    if scaler is not None:
        num_steps.append(("scaler", scaler))

    numeric_transformer = Pipeline(steps=num_steps)
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )
    return preprocess

targets_present = present_targets if present_targets else ["TCH"]  # fallback si sólo hay TCH
targets_present

## 10) Modelado por objetivo — OLS, Ridge, Lasso, KNN, Árbol, RandomForest y Polinomial

In [ ]:

all_results = {}
for target in targets_present:
    print("\n" + "="*70)
    print("Objetivo:", target)
    print("="*70)
    X = df_feat.drop(columns=[target], errors="ignore")
    y = df_feat[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    preprocess = build_preprocess(X_train, scaler_kind=SCALER)

    def eval_model(model, name):
        pipe = Pipeline([("prep", preprocess), ("model", model)])
        cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
        cv_rmse = -cross_val_score(pipe, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error")
        pipe.fit(X_train, y_train)
        preds = pipe.predict(X_test)
        metrics = {
            "MAE": mean_absolute_error(y_test, preds),
            "RMSE": mean_squared_error(y_test, preds, squared=False),
            "R2": r2_score(y_test, preds),
            "cv_RMSE_mean": cv_rmse.mean(),
            "cv_RMSE_std": cv_rmse.std()
        }
        print(f"\n— {name} —")
        display(pd.Series(metrics))
        plt.figure()
        plt.scatter(y_test, preds, alpha=0.6)
        lims = [min(np.min(y_test), np.min(preds)), max(np.max(y_test), np.max(preds))]
        plt.plot(lims, lims, "--")
        plt.xlabel("Real"); plt.ylabel("Predicho"); plt.title(f"Real vs Predicho — {name}")
        plt.show()
        return pipe, metrics

    results = {}
    fitted = {}

    models = [
        (LinearRegression(), "OLS"),
        (Ridge(alpha=1.0, random_state=RANDOM_STATE), "Ridge(alpha=1)"),
        (Lasso(alpha=0.001, random_state=RANDOM_STATE, max_iter=10000), "Lasso(alpha=0.001)"),
        (KNeighborsRegressor(n_neighbors=5), "KNNReg(k=5)"),
        (DecisionTreeRegressor(random_state=RANDOM_STATE), "DecisionTreeReg"),
        (RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE), "RandomForestReg"),
    ]
    for m, name in models:
        p, met = eval_model(m, name)
        results[name] = met
        fitted[name] = p

    # Polinomial (grado 2)
    poly_pipe = Pipeline([("prep", preprocess),
                          ("poly", PolynomialFeatures(degree=2, include_bias=False)),
                          ("model", LinearRegression())])
    cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    rmse_cv = -cross_val_score(poly_pipe, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error")
    poly_pipe.fit(X_train, y_train)
    preds = poly_pipe.predict(X_test)
    metrics_poly = {
        "MAE": mean_absolute_error(y_test, preds),
        "RMSE": mean_squared_error(y_test, preds, squared=False),
        "R2": r2_score(y_test, preds),
        "cv_RMSE_mean": rmse_cv.mean(),
        "cv_RMSE_std": rmse_cv.std()
    }
    print("\n— Polynomial(2) —")
    display(pd.Series(metrics_poly))
    results["Polynomial2"] = metrics_poly
    fitted["Polynomial2"] = poly_pipe

    res_df = pd.DataFrame(results).T.sort_values("RMSE", ascending=True)
    display(res_df)
    all_results[target] = res_df

    # Exportar mejor
    best_name = res_df.index[0]
    joblib.dump(fitted[best_name], os.path.join(OUTPUT_DIR, f"best_pipeline_{target}_{best_name}.joblib"))
    with open(os.path.join(OUTPUT_DIR, f"metrics_{target}.json"), "w", encoding="utf-8") as f:
        json.dump({k:{kk:float(vv) for kk,vv in d.items()} for k,d in results.items()}, f, indent=2)

print("Artefactos guardados en:", OUTPUT_DIR)

## 11) (Opcional) GridSearch y curvas de aprendizaje para el mejor modelo

In [ ]:

for target, res_df in all_results.items():
    print("\n", "#"*20, target, "#"*20)
    best_name = res_df.index[0]
    print("Mejor por RMSE:", best_name)
    # Ejemplo: tunear RF si fue el mejor
    if "RandomForestReg" in best_name:
        X = df_feat.drop(columns=[target], errors="ignore"); y = df_feat[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
        preprocess = build_preprocess(X_train, scaler_kind=SCALER)
        base = Pipeline([("prep", preprocess), ("model", RandomForestRegressor(random_state=RANDOM_STATE))])
        grid = GridSearchCV(
            base,
            param_grid={"model__n_estimators":[200,400], "model__max_depth":[None, 10, 20]},
            scoring="neg_root_mean_squared_error",
            cv=KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE),
            n_jobs=-1
        )
        grid.fit(X_train, y_train)
        print("Best params:", grid.best_params_)
        print("Best CV RMSE:", (-grid.best_score_)**0.5 if grid.best_score_<0 else -grid.best_score_)

        # Curva de aprendizaje
        train_sizes, train_scores, test_scores = learning_curve(
            grid.best_estimator_, X, y, cv=N_FOLDS,
            scoring="neg_root_mean_squared_error",
            n_jobs=-1, train_sizes=np.linspace(0.1,1.0,5), shuffle=True, random_state=RANDOM_STATE
        )
        tr = -train_scores.mean(axis=1)
        te = -test_scores.mean(axis=1)
        plt.figure()
        plt.plot(train_sizes, tr, marker="o", label="train RMSE")
        plt.plot(train_sizes, te, marker="s", label="cv RMSE")
        plt.xlabel("Tamaño de entrenamiento"); plt.ylabel("RMSE")
        plt.title(f"Curva de aprendizaje — {target}")
        plt.legend(); plt.grid(True); plt.show()